In [61]:
import torch
import torch.nn as nn
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

In [3]:
net = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)


net.fc = Linear(in_features=512, out_features=10)

In [4]:
# Data Transformer
tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 1

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=tfm)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=tfm)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(trainloader.dataset)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer=Adam(net.parameters(), lr=0.0003, weight_decay=0.0001)

In [41]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [8]:
def trainer(model,trainloader,min):
  for epoch in range(1):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
      if i>=min and i<=min+4000:
          inputs, labels = data
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()


          running_loss += loss.item()
          if i % 500 == 0:    
              print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 500:.3f}')
              running_loss = 0.0
      elif i>min+4000:
        break
      else:
        pass

  print('Finished Training')

In [43]:
trainer(net,trainloader,0)

[1,     1] loss: 0.006
[1,   501] loss: 2.408
[1,  1001] loss: 2.353
[1,  1501] loss: 2.349
[1,  2001] loss: 2.263
[1,  2501] loss: 2.235
[1,  3001] loss: 2.188
[1,  3501] loss: 2.131
[1,  4001] loss: 2.120
Finished Training


In [44]:
trainer(net,trainloader,4000)

[1,  4001] loss: 0.004
[1,  4501] loss: 2.131
[1,  5001] loss: 2.081
[1,  5501] loss: 2.048
[1,  6001] loss: 2.069
[1,  6501] loss: 2.053
[1,  7001] loss: 1.947
[1,  7501] loss: 2.045
[1,  8001] loss: 1.955
Finished Training


In [45]:
trainer(net,trainloader,8000)

[1,  8001] loss: 0.004
[1,  8501] loss: 2.004
[1,  9001] loss: 1.902
[1,  9501] loss: 1.964
[1, 10001] loss: 1.936
[1, 10501] loss: 1.904
[1, 11001] loss: 1.862
[1, 11501] loss: 1.790
[1, 12001] loss: 1.752
Finished Training


In [46]:
trainer(net,trainloader,12000)

[1, 12001] loss: 0.003
[1, 12501] loss: 1.781
[1, 13001] loss: 1.723
[1, 13501] loss: 1.812
[1, 14001] loss: 1.714
[1, 14501] loss: 1.766
[1, 15001] loss: 1.734
[1, 15501] loss: 1.706
[1, 16001] loss: 1.702
Finished Training


In [47]:
trainer(net,trainloader,16000)

[1, 16001] loss: 0.006
[1, 16501] loss: 1.691
[1, 17001] loss: 1.593
[1, 17501] loss: 1.655
[1, 18001] loss: 1.559
[1, 18501] loss: 1.645
[1, 19001] loss: 1.631
[1, 19501] loss: 1.593
[1, 20001] loss: 1.579
Finished Training


In [48]:
trainer(net,trainloader,20000)

[1, 20001] loss: 0.012
[1, 20501] loss: 1.563
[1, 21001] loss: 1.479
[1, 21501] loss: 1.489
[1, 22001] loss: 1.535
[1, 22501] loss: 1.453
[1, 23001] loss: 1.524
[1, 23501] loss: 1.497
[1, 24001] loss: 1.490
Finished Training


In [49]:
trainer(net,trainloader,24000)

[1, 24001] loss: 0.003
[1, 24501] loss: 1.404
[1, 25001] loss: 1.409
[1, 25501] loss: 1.450
[1, 26001] loss: 1.369
[1, 26501] loss: 1.379
[1, 27001] loss: 1.322
[1, 27501] loss: 1.403
[1, 28001] loss: 1.421
Finished Training


In [50]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [51]:
trainer(net,trainloader,28000)

[1, 28001] loss: 0.007
[1, 28501] loss: 1.395
[1, 29001] loss: 1.322
[1, 29501] loss: 1.356
[1, 30001] loss: 1.351
[1, 30501] loss: 1.307
[1, 31001] loss: 1.233
[1, 31501] loss: 1.268
[1, 32001] loss: 1.249
Finished Training


In [52]:
trainer(net,trainloader,32000)

[1, 32001] loss: 0.003
[1, 32501] loss: 1.291
[1, 33001] loss: 1.248
[1, 33501] loss: 1.244
[1, 34001] loss: 1.204
[1, 34501] loss: 1.325
[1, 35001] loss: 1.207
[1, 35501] loss: 1.181
[1, 36001] loss: 1.227
Finished Training


In [53]:
trainer(net,trainloader,36000)

[1, 36001] loss: 0.002
[1, 36501] loss: 1.268
[1, 37001] loss: 1.230
[1, 37501] loss: 1.246
[1, 38001] loss: 1.202
[1, 38501] loss: 1.160
[1, 39001] loss: 1.162
[1, 39501] loss: 1.239
[1, 40001] loss: 1.147
Finished Training


In [54]:
trainer(net,trainloader,40000)

[1, 40001] loss: 0.007
[1, 40501] loss: 1.146
[1, 41001] loss: 1.159
[1, 41501] loss: 1.118
[1, 42001] loss: 1.101
[1, 42501] loss: 1.212
[1, 43001] loss: 1.122
[1, 43501] loss: 1.057
[1, 44001] loss: 1.126
Finished Training


In [55]:
trainer(net,trainloader,44000)

[1, 44001] loss: 0.002
[1, 44501] loss: 1.141
[1, 45001] loss: 1.168
[1, 45501] loss: 1.083
[1, 46001] loss: 1.131
[1, 46501] loss: 1.140
[1, 47001] loss: 1.018
[1, 47501] loss: 1.071
[1, 48001] loss: 1.177
Finished Training


In [56]:
trainer(net,trainloader,48000)

[1, 48001] loss: 0.000
[1, 48501] loss: 1.028
[1, 49001] loss: 1.096
[1, 49501] loss: 1.021
Finished Training


In [57]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [58]:
trainer(net,trainloader,0)
trainer(net,trainloader,4000)

[1,     1] loss: 0.000
[1,   501] loss: 1.049
[1,  1001] loss: 0.983
[1,  1501] loss: 1.089
[1,  2001] loss: 1.020
[1,  2501] loss: 1.080
[1,  3001] loss: 0.915
[1,  3501] loss: 1.126
[1,  4001] loss: 0.996
Finished Training
[1,  4001] loss: 0.000
[1,  4501] loss: 1.012
[1,  5001] loss: 1.068
[1,  5501] loss: 1.076
[1,  6001] loss: 0.953
[1,  6501] loss: 0.992
[1,  7001] loss: 0.985
[1,  7501] loss: 1.090
[1,  8001] loss: 0.950
Finished Training


In [59]:
trainer(net,trainloader,8000)
trainer(net,trainloader,12000)

[1,  8001] loss: 0.001
[1,  8501] loss: 0.939
[1,  9001] loss: 0.912
[1,  9501] loss: 0.965
[1, 10001] loss: 1.010
[1, 10501] loss: 0.931
[1, 11001] loss: 0.847
[1, 11501] loss: 0.968
[1, 12001] loss: 0.934
Finished Training
[1, 12001] loss: 0.002
[1, 12501] loss: 0.946
[1, 13001] loss: 0.984
[1, 13501] loss: 0.955
[1, 14001] loss: 0.942
[1, 14501] loss: 0.913
[1, 15001] loss: 0.928
[1, 15501] loss: 0.918
[1, 16001] loss: 0.952
Finished Training


In [60]:
trainer(net,trainloader,16000)
trainer(net,trainloader,20000)

[1, 16001] loss: 0.000
[1, 16501] loss: 0.975
[1, 17001] loss: 0.963
[1, 17501] loss: 0.891
[1, 18001] loss: 1.011
[1, 18501] loss: 0.867
[1, 19001] loss: 0.927
[1, 19501] loss: 0.892
[1, 20001] loss: 0.971
Finished Training
[1, 20001] loss: 0.004
[1, 20501] loss: 0.872
[1, 21001] loss: 0.779
[1, 21501] loss: 0.937
[1, 22001] loss: 0.913
[1, 22501] loss: 0.854
[1, 23001] loss: 0.862
[1, 23501] loss: 0.859
[1, 24001] loss: 0.833
Finished Training


In [61]:
trainer(net,trainloader,24000)
trainer(net,trainloader,28000)

[1, 24001] loss: 0.000
[1, 24501] loss: 0.937
[1, 25001] loss: 0.772
[1, 25501] loss: 0.886
[1, 26001] loss: 0.928
[1, 26501] loss: 0.918
[1, 27001] loss: 0.854
[1, 27501] loss: 0.809
[1, 28001] loss: 0.869
Finished Training
[1, 28001] loss: 0.003
[1, 28501] loss: 0.810
[1, 29001] loss: 0.784
[1, 29501] loss: 0.875
[1, 30001] loss: 0.849
[1, 30501] loss: 0.810
[1, 31001] loss: 0.756
[1, 31501] loss: 0.968
[1, 32001] loss: 0.764
Finished Training


In [62]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [63]:
trainer(net,trainloader,32000)
trainer(net,trainloader,36000)

[1, 32001] loss: 0.001
[1, 32501] loss: 0.813
[1, 33001] loss: 0.789
[1, 33501] loss: 0.826
[1, 34001] loss: 0.857
[1, 34501] loss: 0.787
[1, 35001] loss: 0.911
[1, 35501] loss: 0.766
[1, 36001] loss: 0.796
Finished Training
[1, 36001] loss: 0.005
[1, 36501] loss: 0.835
[1, 37001] loss: 0.823
[1, 37501] loss: 0.839
[1, 38001] loss: 0.884
[1, 38501] loss: 0.786
[1, 39001] loss: 0.711
[1, 39501] loss: 0.728
[1, 40001] loss: 0.850
Finished Training


In [64]:
trainer(net,trainloader,40000)
trainer(net,trainloader,44000)

[1, 40001] loss: 0.001
[1, 40501] loss: 0.758
[1, 41001] loss: 0.816
[1, 41501] loss: 0.772
[1, 42001] loss: 0.768
[1, 42501] loss: 0.790
[1, 43001] loss: 0.775
[1, 43501] loss: 0.816
[1, 44001] loss: 0.795
Finished Training
[1, 44001] loss: 0.002
[1, 44501] loss: 0.681
[1, 45001] loss: 0.793
[1, 45501] loss: 0.747
[1, 46001] loss: 0.747
[1, 46501] loss: 0.785
[1, 47001] loss: 0.820
[1, 47501] loss: 0.755
[1, 48001] loss: 0.801
Finished Training


In [65]:
trainer(net,trainloader,48000)

[1, 48001] loss: 0.000
[1, 48501] loss: 0.733
[1, 49001] loss: 0.701
[1, 49501] loss: 0.730
Finished Training


In [66]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [ ]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [ ]:
trainer(net,trainloader,16000)
trainer(net,trainloader,20000)
trainer(net,trainloader,24000)
trainer(net,trainloader,28000)

In [ ]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [ ]:
trainer(net,trainloader,32000)
trainer(net,trainloader,36000)
trainer(net,trainloader,40000)
trainer(net,trainloader,44000)

In [ ]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [ ]:
trainer(net,trainloader,48000)

In [ ]:
trainer(net,trainloader,0)
trainer(net,trainloader,4000)
trainer(net,trainloader,8000)
trainer(net,trainloader,12000)

In [ ]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [71]:
correct = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(trainloader, 0):
        if i<=10000:

          images, labels = data

          outputs = net(images)

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
        else:
          break

print(f'Accuracy of the network on the 10000 train images: {100 * correct // total} %')

Accuracy of the network on the 10000 train images: 73 %


In [72]:
correct = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        if i<=10000:

          images, labels = data

          outputs = net(images)

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
        else:
          break

print(f'Accuracy of the network on the 1000 test images: {100 * correct // total} %')

Accuracy of the network on the 1000 test images: 71 %


In [6]:
net = torch.jit.load('model_scripted.pt')

In [9]:
trainer(net,trainloader,0)
trainer(net,trainloader,4000)
trainer(net,trainloader,8000)
trainer(net,trainloader,12000)

[1,     1] loss: 0.000
[1,   501] loss: 0.694
[1,  1001] loss: 0.725
[1,  1501] loss: 0.682
[1,  2001] loss: 0.694
[1,  2501] loss: 0.780
[1,  3001] loss: 0.784
[1,  3501] loss: 0.699
[1,  4001] loss: 0.752
Finished Training
[1,  4001] loss: 0.001
[1,  4501] loss: 0.666
[1,  5001] loss: 0.751
[1,  5501] loss: 0.711
[1,  6001] loss: 0.721
[1,  6501] loss: 0.703
[1,  7001] loss: 0.749
[1,  7501] loss: 0.669
[1,  8001] loss: 0.697
Finished Training
[1,  8001] loss: 0.000
[1,  8501] loss: 0.675
[1,  9001] loss: 0.749
[1,  9501] loss: 0.717
[1, 10001] loss: 0.707
[1, 10501] loss: 0.816
[1, 11001] loss: 0.665
[1, 11501] loss: 0.732
[1, 12001] loss: 0.674
Finished Training
[1, 12001] loss: 0.000
[1, 12501] loss: 0.739
[1, 13001] loss: 0.656
[1, 13501] loss: 0.790
[1, 14001] loss: 0.681
[1, 14501] loss: 0.715
[1, 15001] loss: 0.629
[1, 15501] loss: 0.782
[1, 16001] loss: 0.717
Finished Training


In [10]:
trainer(net,trainloader,16000)
trainer(net,trainloader,20000)
trainer(net,trainloader,24000)
trainer(net,trainloader,28000)

[1, 16001] loss: 0.003
[1, 16501] loss: 0.716
[1, 17001] loss: 0.718
[1, 17501] loss: 0.740
[1, 18001] loss: 0.695
[1, 18501] loss: 0.652
[1, 19001] loss: 0.728
[1, 19501] loss: 0.727
[1, 20001] loss: 0.756
Finished Training
[1, 20001] loss: 0.000
[1, 20501] loss: 0.727
[1, 21001] loss: 0.717
[1, 21501] loss: 0.747
[1, 22001] loss: 0.718
[1, 22501] loss: 0.750
[1, 23001] loss: 0.650
[1, 23501] loss: 0.789
[1, 24001] loss: 0.726
Finished Training
[1, 24001] loss: 0.000
[1, 24501] loss: 0.658
[1, 25001] loss: 0.761
[1, 25501] loss: 0.720
[1, 26001] loss: 0.626
[1, 26501] loss: 0.708
[1, 27001] loss: 0.680
[1, 27501] loss: 0.751
[1, 28001] loss: 0.708
Finished Training
[1, 28001] loss: 0.000
[1, 28501] loss: 0.671
[1, 29001] loss: 0.750
[1, 29501] loss: 0.646
[1, 30001] loss: 0.712
[1, 30501] loss: 0.750
[1, 31001] loss: 0.731
[1, 31501] loss: 0.745
[1, 32001] loss: 0.701
Finished Training


In [11]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [12]:
trainer(net,trainloader,32000)
trainer(net,trainloader,36000)
trainer(net,trainloader,40000)
trainer(net,trainloader,44000)

[1, 32001] loss: 0.000
[1, 32501] loss: 0.721
[1, 33001] loss: 0.713
[1, 33501] loss: 0.748
[1, 34001] loss: 0.766
[1, 34501] loss: 0.751
[1, 35001] loss: 0.756
[1, 35501] loss: 0.738
[1, 36001] loss: 0.745
Finished Training
[1, 36001] loss: 0.000
[1, 36501] loss: 0.772
[1, 37001] loss: 0.699
[1, 37501] loss: 0.659
[1, 38001] loss: 0.628
[1, 38501] loss: 0.764
[1, 39001] loss: 0.714
[1, 39501] loss: 0.633
[1, 40001] loss: 0.759
Finished Training
[1, 40001] loss: 0.001
[1, 40501] loss: 0.733
[1, 41001] loss: 0.831
[1, 41501] loss: 0.719
[1, 42001] loss: 0.686
[1, 42501] loss: 0.635
[1, 43001] loss: 0.737
[1, 43501] loss: 0.753
[1, 44001] loss: 0.698
Finished Training
[1, 44001] loss: 0.000
[1, 44501] loss: 0.780
[1, 45001] loss: 0.782
[1, 45501] loss: 0.753
[1, 46001] loss: 0.736
[1, 46501] loss: 0.772
[1, 47001] loss: 0.701
[1, 47501] loss: 0.769
[1, 48001] loss: 0.720
Finished Training


In [13]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

In [14]:
#test2

In [15]:
correct = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(trainloader, 0):
        if i<=10000:

          images, labels = data

          outputs = net(images)

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
        else:
          break

print(f'Accuracy of the network on the 10000 train images: {100 * correct // total} %')

Accuracy of the network on the 10000 train images: 74 %


In [16]:
correct = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        if i<=10000:

          images, labels = data

          outputs = net(images)

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
        else:
          break

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 73 %


In [60]:
net2 = resnet18(weights=ResNet18_Weights.DEFAULT)


net2.fc = Linear(in_features=512, out_features=10)

In [56]:
# Data Transformer
tfm = tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 1

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=tfm)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=tfm)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(trainloader.dataset)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [59]:
correct = 0
total = 0

with torch.no_grad():
    for i,data in enumerate(testloader,0):
        

          images, labels = data

          outputs = net2(images)

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

print(f'FOR PRETRAINED RESNET18 model, Accuracy of the network on the 10000 train images: {100 * correct // total} %')

FOR PRETRAINED RESNET18 model, Accuracy of the network on the 10000 train images: 10 %
